In [1]:
%run ../scripts/notebook_settings_lean.py
from scipy import stats
from horizonplot import horizonplot
from chromwindow import window
import zarr
import allel

Loading metadata

In [2]:
zarr_chrX_dir = "/home/eriks/baboondiversity/data/PG_panu3_zarr_12_03_2021/callset.zarr/chrX"
#Opening the zarr data
callset_f = zarr.open_group(zarr_chrX_dir, mode="r")
gt_f = allel.GenotypeArray(callset_f["calldata/GT"])
pos_f = callset_f["variants/POS"][:]

zarr_all_chrX_dir = "/home/eriks/baboondiversity/data/PG_panu3_zarr_12_03_2021/callset.zarr/all_chrX"
#Opening the zarr data
callset_all_chrX = zarr.open_group(zarr_all_chrX_dir, mode="r")
gt_all_chrX = allel.GenotypeArray(callset_all_chrX["calldata/GT"])
pos_all_chrX = callset_all_chrX["variants/POS"][:]

zarr_dipmale_chrX = "/home/eriks/baboondiversity/data/PG_panu3_zarr_12_03_2021/callset.zarr/dipmale_chrX"
#Opening the zarr data
callset_dipmale_chrX = zarr.open_group(zarr_dipmale_chrX, mode="r")

In [3]:
hap_starter = pd.read_csv("../data/haploid_metadata.txt", sep =" ")
meta_data_samples_sci = pd.read_csv("../data/Papio_metadata_with_clustering_sci.txt", sep =" ")
meta_data_samples = pd.read_csv("../data/Papio_metadata_with_clustering.txt", sep =" ")

#Generate a mapping between metadata and callset - repeat for chrX females to be sure of no errors.
ID_to_callset = dict(zip(callset_all_chrX["samples"][:], range(len(callset_all_chrX["samples"][:]))))
hap_meta_data_samples = hap_starter.loc[hap_starter.PGDP_ID.isin(callset_all_chrX["samples"][:])].copy()
hap_meta_data_samples["callset_index"] = hap_meta_data_samples.PGDP_ID.map(ID_to_callset)

ID_to_callset_f = dict(zip(callset_f["samples"][:], range(len(callset_f["samples"][:]))))
meta_data_samples_f = meta_data_samples_sci.loc[meta_data_samples_sci.PGDP_ID.isin(callset_f["samples"][:])].copy()
meta_data_samples_f["callset_index"] = meta_data_samples_f.PGDP_ID.map(ID_to_callset_f)

ID_to_callset_dip = dict(zip(callset_dipmale_chrX["samples"][:], range(len(callset_dipmale_chrX["samples"][:]))))
meta_data_samples_dip = meta_data_samples_sci.loc[meta_data_samples_sci.PGDP_ID.isin(callset_dipmale_chrX["samples"][:])].copy()
meta_data_samples_dip["callset_index"] = meta_data_samples_dip.PGDP_ID.map(ID_to_callset_dip)

In [4]:
rfmix_path = "/home/eriks/baboondiversity/people/eriks/second_analysis_baboons/steps/rfmix_gen100/tanzania_focus/"
df_l = []
chroms = ["chr{}".format(x) for x in (range(1, 21))]+["all_chrX", "female_chrX"]
for c in chroms:
    read_file = rfmix_path + "{}.windows.txt".format(c)
    df = pd.read_csv(read_file, sep="\t")
    df_l.append(df)
window_df_tanz = pd.concat(df_l)
mean_window_df_tanz = window_df_tanz.groupby(["chrom", "individual", "start", "end"]).mean().reset_index()
tanz_olives = meta_data_samples_sci.loc[meta_data_samples_sci.C_origin == "Anubis, Tanzania"].PGDP_ID
mean_window_df_tanz_olive = mean_window_df_tanz.loc[mean_window_df_tanz.individual.isin(tanz_olives)]

In [7]:
focus_pop_tag = "Anubis, Tanzania"
donor_1 = "Anubis, Ethiopia"
donor_2 = "Cynocephalus, Central Tanzania"
window_size = 100000
all_chroms = ["chrX"] # ["chr{}".format(x) for x in (range(1, 21))]+["all_chrX", "chrX", "dipmale_chrX"]

def fst(ac1, ac2):
    num, den = allel.hudson_fst(ac1, ac2, fill=0)
    return np.sum(num) / np.sum(den)

c_df_l = []
for c in all_chroms:
    print(c)
    zarr_dir = "/home/eriks/baboondiversity/data/PG_panu3_zarr_12_03_2021/callset.zarr/" + c
    #Opening the zarr data
    callset = zarr.open_group(zarr_dir, mode="r")
    gt = allel.GenotypeArray(callset["calldata/GT"])
    pos = callset["variants/POS"][:]
    # Loading in the IDs and gt
    if c == "all_chrX":
        metadata = hap_meta_data_samples
    elif c == "chrX":
        metadata = meta_data_samples_f
    elif c == "dipmale_chrX":
        metadata = meta_data_samples_dip
    else:
        metadata = meta_data_samples
    focus_pop_IDs = metadata.loc[metadata.C_origin == focus_pop_tag].PGDP_ID
    focus_pop_gt = gt.take(metadata.loc[metadata.C_origin == focus_pop_tag].callset_index,
                       axis=1)
    donor_1_pop = metadata.loc[metadata.C_origin == donor_1].callset_index
    donor_1_gt = gt.take(donor_1_pop, axis=1)
    donor_2_pop = metadata.loc[metadata.C_origin == donor_2].callset_index
    donor_2_gt = gt.take(donor_2_pop, axis=1)
    df_l = []
    for ind in focus_pop_IDs:
        if ind[-1:] == "b":
            continue
        elif ind[-1:] == "a":
            focus_ind_gt = focus_pop_gt = gt.take(metadata.loc[metadata.PGDP_ID.isin([ind, ind[:-1]+"b"])].callset_index,
                           axis=1)
            rest_pop_gt = focus_pop_gt = gt.take(metadata.loc[(~metadata.PGDP_ID.isin([ind, ind[:-1]+"b"])) &
                                                                                   metadata.PGDP_ID.isin(focus_pop_IDs)].callset_index,
                           axis=1)
            ID = ind[:-2]
        else:
            focus_ind_gt = focus_pop_gt = gt.take(metadata.loc[metadata.PGDP_ID.isin([ind])].callset_index,
                           axis=1)
            rest_pop_gt = focus_pop_gt = gt.take(metadata.loc[(~metadata.PGDP_ID.isin([ind])) &
                                                                               metadata.PGDP_ID.isin(focus_pop_IDs)].callset_index,
                           axis=1)
            ID = ind
        if ID[0] != "P" or ID[0] != "S":
            ID = "Sci_" + ID
        dxy_focus, windows, n_bases, counts =  allel.windowed_divergence(pos, focus_ind_gt.count_alleles(),
                          rest_pop_gt.count_alleles(), size=window_size, start=0)
        dxy_donor_1, windows, n_bases, counts =  allel.windowed_divergence(pos, focus_ind_gt.count_alleles(),
                          donor_1_gt.count_alleles(), size=window_size, start=0)
        dxy_donor_2, windows, n_bases, counts =  allel.windowed_divergence(pos, focus_ind_gt.count_alleles(),
                          donor_2_gt.count_alleles(), size=window_size, start=0)
        df_l.append(pd.DataFrame({"chrom": c, "individual": ID, "start": windows[:,0], "focus_div": dxy_focus,
                  "donor_1_div": dxy_donor_1, "donor_2_div": dxy_donor_2}))
    focus_df = pd.concat(df_l)
    if c == "chrX":
        rf_c = "female_chrX"
    elif c == "dipmale_chrX":
        rf_c = "all_chrX"
    else:
        rf_c = c
    print(c, rf_c)
    c_rfmix = mean_window_df_tanz_olive.loc[mean_window_df_tanz_olive.chrom == rf_c]
    c_rfmix["chrom"] = c
    per_ind_df = c_rfmix.merge(focus_df, on=["chrom", "individual", "start"])
    pi, windows, n_bases, counts_pi = allel.windowed_diversity(pos,
                                                        ac=focus_pop_gt.count_alleles(), size=window_size, start=0)
    fst1, windows, counts = allel.windowed_statistic(pos, (focus_pop_gt.count_alleles(),
                                                      donor_1_gt.count_alleles()), fst, size=window_size, start=0)
    fst2, windows, counts = allel.windowed_statistic(pos, (focus_pop_gt.count_alleles(),
                                                      donor_2_gt.count_alleles()), fst, size=window_size, start=0)
    theta_hat_w, windows, n_bases, counts = allel.windowed_watterson_theta(pos,
                                                        ac=focus_pop_gt.count_alleles(), size=window_size, start=0)

    group_stats = pd.DataFrame({"chrom": c, "start": windows[:,0], "pi": pi, "count": counts_pi,
                            "fst_1": fst1, "fst_2": fst2, "w_theta": theta_hat_w})
    ind_group_df = per_ind_df.merge(group_stats, on=["chrom", "start"])
    ind_group_df.to_csv("../steps/rfmix_stats_df/{}_stats_tanz_anubis.txt".format(c), index=False)

chrX
chrX female_chrX


/scratch/25546149/ipykernel_1957803/933225791.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_rfmix["chrom"] = c
